In [24]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import scipy.io

# Load the .mat file
mat = scipy.io.loadmat('bike/Codes/Codes/Check-out Proportion/fea_pro.mat')

# Define PyTorch dataset class
class BikeDataset(Dataset):
    def __init__(self,H,P):
        """
        H: amount of historical timestamp
        P: amount of predicted timestamp
        X: historical data with shape 
        (number of samples, number of historical timestamp, entire traffic + check-out across 23 clusters + check-out proportion across 23 clusters + 7 features)
        Y: future data with shape
        (number of samples, number of futur timestamp,check-out across 23 clusters)
        """
        
        # Get the 2D matrix
        fea = mat['fea']
        m_o = mat['m_o']
        m_O = mat['m_O']
        rho = mat['rho']
        
        # Split the data into input and output features
        fea_H = []
        m_o_H = []
        m_o_P = []
        m_O_H = []
        rho_H = []
        for i in range(H, len(fea) - P + 1):
            fea_H.append(fea[i-H:i])
            m_o_H.append(m_o[i-H:i])
            m_o_P.append(m_o[i:i+P])
            m_O_H.append(m_O[0][i-H:i])
            rho_H.append(rho[i-H:i])
        fea_H = torch.tensor(fea_H, dtype=torch.float32)
        m_o_H = torch.tensor(m_o_H, dtype=torch.float32)
        m_O_H = torch.tensor(m_O_H, dtype=torch.float32)
        m_O_H = m_O_H.unsqueeze(-1)
        rho_H = torch.tensor(rho_H, dtype=torch.float32)
        
        # Concetenate four tensors
        X = torch.cat([m_O_H, m_o_H, rho_H, fea_H], dim=2)
        
        #print(X.shape)
        self.x = X
        self.y = torch.tensor(m_o_P, dtype=torch.float32)

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

# Create PyTorch dataset and data loader
dataset = BikeDataset(3,2)
print(dataset.x.size())
print(dataset.y.size())
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# define neural network model
class BikeModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = torch.nn.Linear(3*(7+23+1+23), 128)
        self.fc2 = torch.nn.Linear(128, 64)
        self.fc3 = torch.nn.Linear(64, 2*23)
    
    def forward(self, x):
        x = x.view(x.shape[0], -1)  # flatten the input tensor
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x.view(x.shape[0], 2, 23)  # reshape the output tensor

# define loss function and optimizer
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(BikeModel().parameters())

# train the model
num_epochs = 10
for epoch in range(num_epochs):
    epoch_loss = 100.0
    for batch_idx, (data, target) in enumerate(dataloader):
        optimizer.zero_grad()
        output = BikeModel()(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch {epoch+1} loss: {epoch_loss/len(dataloader):.4f}")


torch.Size([4388, 3, 54])
torch.Size([4388, 2, 23])
Epoch 1 loss: 7224.3987
Epoch 2 loss: 7292.3802
Epoch 3 loss: 7197.8602
Epoch 4 loss: 7199.7444
Epoch 5 loss: 7190.8045
Epoch 6 loss: 7194.3698
Epoch 7 loss: 7206.4779
Epoch 8 loss: 7220.1123
Epoch 9 loss: 7249.8706
Epoch 10 loss: 7214.2600
